In [1]:
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.llms import GooglePalm
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

In [2]:
api_key = ""

In [3]:
llm = GooglePalm(google_api_key=api_key,temperature=0.6)

In [4]:
from langchain.utilities import SQLDatabase
db_user = "root"
db_password = ""
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [12]:
db_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True)
qns1 = db_chain.run("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: [(40,)]
Answer:40
> Finished chain.


In [13]:
qns1

'40'

In [14]:
qns2 = db_chain.run("How much is the price of inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of inventory for all small size t-shirts?
SQLQuery:SELECT SUM(price) AS inventory_price FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('407'),)]
Answer:407
> Finished chain.


In [15]:
qns2 = db_chain.run("select sum(price*stock_quantity) from t_shirts where size ='S'")



> Entering new SQLDatabaseChain chain...
select sum(price*stock_quantity) from t_shirts where size ='S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('19692'),)]
Answer:19692
> Finished chain.


In [16]:
qns2 = db_chain.run("If we have to sell all the Levi' s T-shirts today with discount applied How much revenue our store will generate (post discounts)?") 



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levi' s T-shirts today with discount applied How much revenue our store will generate (post discounts)?
SQLQuery:SELECT SUM(price*(1-pct_discount)) FROM t_shirts JOIN discounts ON t_shirts.t_shirt_id=discounts.t_shirt_id WHERE brand='Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'discounts.start_date' in 'where clause'")
[SQL: SELECT SUM(price*(1-pct_discount)) FROM t_shirts JOIN discounts ON t_shirts.t_shirt_id=discounts.t_shirt_id WHERE brand='Levi' AND CURDATE() BETWEEN discounts.start_date AND discounts.end_date]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [17]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
"""
qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id

SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('13509.000000'),)]
Answer:13509
> Finished chain.


In [18]:
qns3

'13509'

In [19]:
qns4 = db_chain.run("select sum(price * stock_quantity) from t_shirts where brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
select sum(price * stock_quantity) from t_shirts where brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('15065'),)]
Answer:15065
> Finished chain.


In [20]:
qns5 = db_chain.run("How many white color Levi's t-shirts we have available?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's t-shirts we have available?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(10,), (10,), (12,)]
Answer:12
> Finished chain.


In [21]:
qns5 = db_chain.run("select sum(stock_quantity) from t_shirts where brand='Levi' and color='White'")



> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) from t_shirts where brand='Levi' and color='White'
SQLQuery:SELECT sum(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White'
SQLResult: [(Decimal('32'),)]
Answer:32
> Finished chain.


In [65]:
qns1 = db_chain.run("select sum(stock_quantity) from t_shirts where brand='Nike' and size='XS' and color = 'White'")
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")




> Entering new SQLDatabaseChain chain...
select sum(stock_quantity) from t_shirts where brand='Nike' and size='XS' and color = 'White'
SQLQuery:SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Nike' AND `size` = 'XS' AND `color` = 'White'
SQLResult: [(Decimal('40'),)]
Answer:40
> Finished chain.


> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('19692'),)]
Answer:19692
> Finished chain.


> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(pr

In [66]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns1},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]


In [67]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceBgeEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

e = embeddings.embed_query("How many white color Levi' s shirt I have?")

In [68]:
len(e)

384

In [69]:
e[:5]

[0.028084727004170418,
 0.12849904596805573,
 0.005204128101468086,
 0.013358879834413528,
 0.043256618082523346]

In [72]:
[example.values() for example in few_shots]

[dict_values(['How many t-shirts do we have left for Nike in XS size and white color?', "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'", 'Result of the SQL query', '40']),
 dict_values(['How much is the total price of the inventory for all S-size t-shirts?', "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'", 'Result of the SQL query', '19692']),
 dict_values(['If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?', "SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n ", 'Result of the SQL query', '13509']),
 dict_values(['If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate witho

In [73]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [74]:
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 40",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 19692",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 13509",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' Re

In [75]:
to_vectorize[0]

"How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 40"

In [76]:
from langchain.vectorstores import Chroma

In [79]:
vectorstore = Chroma.from_texts(to_vectorize,embedding=embeddings,metadatas=few_shots)

In [88]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore = vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left my store?"})

[{'Answer': '40',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '15065',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

In [96]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [97]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [98]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [106]:
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""


In [107]:
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)    

In [108]:
from langchain.prompts import FewShotPromptTemplate

In [111]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt= example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input","tabel_info","top_k"],
)   

In [113]:
example_prompt

PromptTemplate(input_variables=['Answer', 'Question', 'SQLQuery', 'SQLResult'], template='\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}')

In [114]:
example_selector

SemanticSimilarityExampleSelector(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x0000026C065B62F0>, k=2, example_keys=None, input_keys=None)

In [118]:
new_chain = SQLDatabaseChain.from_llm(llm,db,verbose=True,prompt=few_shot_prompt)
new_chain("How many white color Levi's shirts i have?")



> Entering new SQLDatabaseChain chain...
How many white color Levi's shirts i have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('32'),)]
Answer:32
> Finished chain.


{'query': "How many white color Levi's shirts i have?", 'result': '32'}

In [119]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT sum(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('19692'),)]
Answer:19692
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '19692'}

In [120]:
new_chain("How much is the price of all white color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SELECT sum(price*stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('896'),)]
Answer:896
> Finished chain.


{'query': 'How much is the price of all white color levi t shirts?',
 'result': '896'}

In [121]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('22575.000000'),)]
Answer:22575
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '22575'}

In [122]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Van Huesen'
SQLResult: [(Decimal('27399'),)]
Answer:27399
> Finished chain.


'27399'